In [122]:
from utils.excel_utils import save_to_excel
from machine_learning.sampling.sample import sample_dataset
from sklearn.neural_network import MLPClassifier
from instance_selection.parameter.parameter import *  # 导入参数的设定
from sklearn.model_selection import train_test_split
import scipy.io as sio  # 从.mat文件中读取数据集
import numpy as np
import warnings
from sklearn.base import clone

warnings.filterwarnings("ignore")  # 忽略警告

DATASET = Lymphography  # 数据集名称（包含对应参数的字典形式）
datasetname = DATASET.DATASETNAME.split('.')[0]

# 加载、划分数据集
mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + DATASET.DATASETNAME)
x = mat_data['X']
y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]

num_run = 30  # 运行次数

gmean_results = []
mauc_results = []

model = MLPClassifier(hidden_layer_sizes=(DATASET.HIDDEN_SIZE,), max_iter=DATASET.MAX_ITER,
                      random_state=RANDOM_SEED, learning_rate_init=DATASET.LEARNING_RATE)

for i in range(num_run):
    random_state = RANDOM_SEED + np.random.randint(1, 1000)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=RANDOM_SEED)  # 划分数据集

    gmean_ros, mauc_ros = sample_dataset(clone(model), x_train, x_test, y_train, y_test, random_state, method='ROS')
    gmean_rus, mauc_rus = sample_dataset(clone(model), x_train, x_test, y_train, y_test, random_state, method='RUS')
    gmean_nos, mauc_nos = sample_dataset(clone(model), x_train, x_test, y_train, y_test, random_state, method='NOS')
    gmean_smote, mauc_smote = sample_dataset(clone(model), x_train, x_test, y_train, y_test, random_state,
                                             method='SMOTE')
    # gmean_adasyn, mauc_adasyn = sample_dataset(clone(model), x_train, x_test, y_train, y_test, random_state, method='ADASYN')
    gmean_borderlinesmote1, mauc_borderlinesmote1 = sample_dataset(clone(model), x_train, x_test, y_train, y_test,
                                                                   random_state, method='BorderlineSMOTE')
    gmean_kmeanssmote, mauc_kmeanssmote = sample_dataset(clone(model), x_train, x_test, y_train, y_test, random_state,
                                                         method='KMeansSMOTE')

    gmean = [gmean_nos, gmean_rus, gmean_ros, gmean_smote, gmean_borderlinesmote1, gmean_kmeanssmote]
    mauc = [mauc_nos, mauc_rus, mauc_ros, mauc_smote, mauc_borderlinesmote1, mauc_kmeanssmote]
    gmean_results.append(gmean)
    mauc_results.append(mauc)
    # 输出结果
    print(f'第{i+1}次运行:')
    print(f'Gmean: {gmean}')
    print(f'mAUC: {mauc}')

columns = ['NOS', 'RUS', 'ROS', 'SMOTE', 'BorderlineSMOTE', 'KMeansSMOTE']

save_path = 'C:/Users/zsc/Desktop/experiment/' + datasetname + '/'
filename = datasetname
save_to_excel(save_path, filename, columns, gmean_results, mauc_results)

第1次运行:
Gmean: [0.867204, 0.725969, 0.886062, 0.881423, 0.90059, 0.867204]
mAUC: [0.953261, 0.91558, 0.936232, 0.951087, 0.939493, 0.943841]
第2次运行:
Gmean: [0.867204, 0.0, 0.886062, 0.90059, 0.914664, 0.895197]
mAUC: [0.953261, 0.83442, 0.92971, 0.930072, 0.958333, 0.942029]


RuntimeError: No clusters found with sufficient samples of class 0. Try lowering the cluster_balance_threshold or increasing the number of clusters.